# FEATURE SELECTION

In [ ]:
#Correlation Plot
corr = predictorsNumerical.corr()
heatmap = sns.heatmap(corr, vmin=0, vmax=1)

In [ ]:
#Removing variables for treating multi-collinearity
predictorsNumerical=predictorsNumerical.drop(['YearsAtCompany','YearsWithCurrManager','TotalWorkingYears'],axis=1)

In [ ]:
predictors = pd.concat([predictorsCategorical.reset_index(drop=True),predictorsNumerical.reset_index(drop=True)],axis=1)

from sklearn.preprocessing import LabelEncoder

# One hot encoding Categorical and Ordinal Variables
predictorsCategorical_encoded  = pd.get_dummies(predictorsCategorical)

# Encoding Target to binary 0 & 1
number = LabelEncoder()
y = number.fit_transform(target.astype('str'))

X = pd.concat([predictorsCategorical_encoded.reset_index(drop=True),predictorsNumerical.reset_index(drop=True)],axis=1)
X.shape

In [ ]:
#VIF
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer

from statsmodels.stats.outliers_influence import variance_inflation_factor

from statsmodels.stats.outliers_influence import variance_inflation_factor

class ReduceVIF(BaseEstimator, TransformerMixin):
    def __init__(self, thresh=10.0, impute=True, impute_strategy='median'):
        # From looking at documentation, values between 5 and 10 are "okay".
        # Above 10 is too high and so should be removed.
        self.thresh = thresh
        
        # The statsmodel function will fail with NaN values, as such we have to impute them.
        # By default we impute using the median value.
        # This imputation could be taken out and added as part of an sklearn Pipeline.
        if impute:
            self.imputer = Imputer(strategy=impute_strategy)

    def fit(self, X, y=None):
        print('ReduceVIF fit')
        if hasattr(self, 'imputer'):
            self.imputer.fit(X)
        return self

    def transform(self, X, y=None):
        print('ReduceVIF transform')
        columns = X.columns.tolist()
        if hasattr(self, 'imputer'):
            X = pd.DataFrame(self.imputer.transform(X), columns=columns)
        return ReduceVIF.calculate_vif(X, self.thresh)

    @staticmethod
    def calculate_vif(X, thresh=10.0):
        # Taken from https://stats.stackexchange.com/a/253620/53565 and modified
        dropped=True
        while dropped:
            variables = X.columns
            dropped = False
            vif = [variance_inflation_factor(X[variables].values, X.columns.get_loc(var)) for var in X.columns]
            
            max_vif = max(vif)
            if max_vif > thresh:
                maxloc = vif.index(max_vif)
                print(f'Dropping {X.columns[maxloc]} with vif={max_vif}')
                X = X.drop([X.columns.tolist()[maxloc]], axis=1)
                dropped=True
        return X

transformer = ReduceVIF()

# Only use 50 columns for speed in this example
X = transformer.fit_transform(X[X.columns[-50:]], y)
X.head()